# Data Processing Notebook
This template script contains all the code needed to import and process FTIR spectra, then save them in a standardised file format. It can import and process a single file, or multiple files at once. Available processing includes:
- Averaging for multi-spec measurements
- Baseline (background) subtraction
- Reference spectrum subtraction
- Automatic peak detection
- Automatic peak fitting

# Requires input data files to be organised as follows:
- Location: {Data_dir}/{Sample}/
- Sample/Measurement metadata must be recorded in either file {spec filename}_MetaData.csv, or within spec filename
    - Spec filename format:
    
        {Spec ID number}_{Sample}_{Subsample}_{notes}.txt
    
    - Each spectrum file in a given project must have a unique identifier (preferable a sequential ID number) so that it can always be distinguished from other measurements, even when they are otherwise identical. These ID numbers can be used to refer to particular spectra when manually specifying settings to use for outliers/exceptional cases.
    - any pre-processing steps applied to the data should be included at the end of the filename
        - C: cosmic ray removal
        - O: outlier removal
        - N: normalisation
        
# How to use this script:
- Pick your directories and specify what sample/setting filters you want to apply when importing files
- Adjust the data import string to make sure it matches your file structure and naming conventions (if necessary)
- Each section of code starts with a set of user input variables to control what it does:

             skip: skip this section entirely (True/False)
            debug: print debug messages during execution (True/False)
        show_plot: show plotted figures in viewer (True/False)
        save_plot: save plotted figures to file (True/False)
        
    Data processing sections also need to be told what x,y values and keynames to use for input/output:
    
            x_key: the keyname to use as x values (e.g. 'raman_shift')
            y_key: the keyname to use as y values (e.g. 'y_av_sub')
          alt_key: the back-up keyname to use as y values if y_key does not exist for a given measurement (e.g. 'y_av')
          new_key: the keyname to save the output under (e.g. 'y_av_refsub' or 'fitted_peaks')
          
    Some processing steps also have manual overrides, for example if you want to specify what measurements need to have a reference spectrum subtracted, or which peak positions to fit for a given measurement. Please refer to those sections.

- To turn off/on sections of code, e.g. peak detection, simply use the 'skip' variable at the start of each section.
- When rerunning sections of code, bear in mind that data and variables will reflect the most recent state and may give you unexpected results - sometimes it's safer to rerun the whole script. 
- When you are happy with the output of a particular section, you can switch the 'debug' variable to False to hide debug text generated in the window. It will continue to show key information, and plots.
- To save time and memory, you can switch 'show_plot' to False to stop plots being rendered in the viewer. They will still be saved to disk if 'save_plot' is True.


In [1]:
# ==================================================
# define where your data is, and what files to import

# list directories for input data, figures and output files
Data_dir = '../FTIR data/'
Fig_dir = '../FTIR figures/'
Out_dir = '../FTIR output/'
Ref_dir = '../FTIR data/References/'

Technique = 'FTIR'         # 'Raman' or 'FTIR'

# filter data import by sample / subsample
Sample = '*'                # name of sample, or '*' for all
Subsample = '*'             # name of subsample, or '*'
Measurement_ID = '*'        # unique ID of measurement, or '*'

# list spectral IDs to skip when importing data, as strings
Do_Not_Import = ['012-JRH']

# Import data by sample or spectrum?
# - 'sample':   imports all spectra with the same sample name at the same time, referenced by sample name
# - 'spectrum': imports spectra individually, referenced by spectrum ID
Import_By = 'spectrum'

# filter by measurement settings
Metadata_File = False       # set to True to import metadata file, or False to extract data from spec filename
Measurement_Date = '*'      # Date in YYYY-MM-DD format as string, or '*'
Preprocessing = '*'         # specify required preprocessing, or '*' for best available

In [2]:
# ==================================================
# this section imports necessary python modules

import os
import math
import glob
import datetime
import numpy as np
import pandas as pd
import lmfit as lmfit
import matplotlib as mpl
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit
from scipy.signal import argrelextrema
from scipy.signal import savgol_filter

# import VibSpec module functions
from OSTRI_functions import *

"""
# ==================================================
# FILE SEARCH
# - this section searches for spectrum files that match the specified settings
# ==================================================
"""

print("SEARCHING FOR SPECTRUM DATA FILES...")

# find files
#   - currently requires data files to be named and organised as follows:
# Data_dir/{Sample}/{Measurement_ID}_{Sample}_{Subsample}_{Measurement Parameters}.txt
text = "%s%s/%s_%s_%s_%s_%s*CSV" % (Data_dir, Measurement_Date, Measurement_ID,
        Polymer, Environment, Timepoint, Sample)
print(text)
spec_dirs = sorted(glob.glob(text))

if Import_By.lower() == 'sample':
    # importing by sample name
    
    Sample_Names = np.unique([spec.split("/")[-1].split("_")[2] for spec in spec_dirs])
    print()
    print("Sample names found:", Sample_Names)
    
    # find all unique measurement IDs for each sample
    spec_dirs = []
    for name in Sample_Names:
        text = "%s%s/*.CSV" % (Data_dir, name)
        temp = sorted(glob.glob(text))
        Spec_IDs = np.unique([spec.split("/")[-1].split("_")[0] for spec in temp])
        for ID in Spec_IDs:
            # find appropriate pre-processed file for this measurement
            spec_dirs.append([])
            if Preprocessing != '*':
                # use specified preprocessing
                text = "%s%s/*_%s.CSV" % (Data_dir, name, ID, Preprocessing)
                temp = sorted(glob.glob(text))
                if len(temp) > 0:
                    lengths = [len(s) for s in temp]
                    spec_dirs[-1].append(temp[np.argmax(lengths)])
            else:
                # use best available
                text = "%s%s/%s_*.CSV" % (Data_dir, Sample, ID)
                temp = sorted(glob.glob(text))
                if len(temp) > 0:
                    lengths = [len(s) for s in temp]
                    spec_dirs[-1].append(temp[np.argmax(lengths)])
    
else:
    # importing by spectrum ID
    
    Spec_IDs = np.unique([spec.split("/")[-1].split("_")[0] for spec in spec_dirs])
    print()
    print("spectrum IDs found:", Spec_IDs)

    # find appropriate pre-processed file for each measurement
    spec_dirs = []
    for ID in Spec_IDs:
        if Preprocessing != '*':
            # use specified preprocessing
            text = "%s%s/%s_*_%s.CSV" % (Data_dir, Sample, ID, Preprocessing)
            temp = sorted(glob.glob(text))
            if len(temp) > 0:
                lengths = [len(s) for s in temp]
                spec_dirs.append(temp[np.argmax(lengths)])
        else:
            # use best available
            text = "%s%s/%s_*.CSV" % (Data_dir, Sample, ID)
            temp = sorted(glob.glob(text))
            if len(temp) > 0:
                lengths = [len(s) for s in temp]
                spec_dirs.append(temp[np.argmax(lengths)])

print()
print("data files found:", len(spec_dirs))
for file in spec_dirs:
    print("    ", file.split("/")[-1])
    

SyntaxError: invalid syntax (OSTARIS_functions.py, line 19)

# Data import

- this section actually imports data files and extracts their metadata
- metadata can be in a separate CSV file with the same name plus '\_metadata', or can be extracted from the data file's name
- measurements are imported file by file, and added sequentially to the 'data' dict
- to access a specific measurement, you call data\[measurement ID\]\[key\], where 'key' is the type of data you want from it

In [ ]:
print("IMPORTING DATA...")

# print debug messages?
debug = True

# set up data storage dictionary
data = {}

# ==================================================
# each measurement imported will be added to this dictionary as a Measurement object
# To access a particular measurement, use data[measurement ID]

# for each detected file
for spec_dir in spec_dirs:
    while True:
        try:
            filename = spec_dir.split("/")[-1][:-4]
            ID = filename.split("_")[0]
            notes = ''
            if ID in Do_Not_Import:
                print()
                print(" measurement %s is in Do_Not_Import list, skipping" % filename)
                break
            else:
                print()
                print("importing %s" % filename)
                # extract sample/measurement metadata
                metadata = False
                if Metadata_File == True:
                    # search for metadata file with matching name
                    metadata_dir = glob.glob("%s_Metadata.csv" % spec_dir[:-4])
                    print("    metadata files found:", len(metadata_dir))
                    if len(metadata_dir) > 0:
                        metadata = True
                if metadata == True:
                    # get metadata from metadata file
                    metadata = pd.read_csv(metadata_dir)
                else:
                    # get metadata from filename instead
                    filename_split = filename.split("_")
                    # get sample info
                    ID, date, sample, subsample = filename_split[:4]
                    date = datetime.datetime.strptime(date, "%Y-%m-%d")
                    # check for other info
                    notes = ''
                    if len(filename_split) >= 6:
                        notes = filename_split[5]
                
                # report metadata
                if debug == True:
                    print("      measurement ID:", ID)
                    print("              sample:", sample)
                    print("           subsample:", subsample)
                    print("         measured on:", date.strftime("%Y-%m-%d"))
                    print("               notes:", notes)
                    print("    measurement settings:")
                    print("                mode: FTIR")
                # import spectrum file (assumes CSV file formatting with 2 columns for x,y)
                spec = np.genfromtxt(spec_dir, delimiter=',').transpose()
                if debug == True:
                    print("    spec array:", np.shape(spec))
                # single point, columns=(raman_shift, intensity)
                spec_type = 'point'
                points = 1
                xy_coords = np.asarray([[0],[0]])
                print("        single point measurement")
                x = spec[0]
                y = spec[1:]
                sort = np.argsort(x)
                x = x[sort]
                y = y.transpose()[sort,:]
                
                # check if transmittance or absorbance
                if np.mean(y) > 10:
                    mode = 'T'
                    ylabel = 'Transmittance (%)'
                else:
                    mode = 'A'
                    ylabel = 'Absorbance (arb. u.)'
                
                # trim infinities from both
                check = np.logical_and(np.isinf(y) == False, np.isnan(y) == False)
                if mode == 'T':
                    # also trim 0's from transmittance (to ensure safe calculation of absorbance)
                    check = np.logical_and(check, y != 0)
                check = np.all(check, axis=1)
                
                print("               mode:", ylabel)
                if debug == True:
                    print("            x array:", np.shape(x))
                    print("            x range: %0.f - %0.f cm-1" % (np.amin(x), np.amax(x)))
                    print("            y array:", np.shape(y))
                    print("            y range: %0.2f - %0.2f" % (np.amin(y), np.amax(y)))
                    print("          inf check:", np.any(np.isinf(y)))
                    print("          nan check:", np.any(np.isnan(y)))
                # generate sample title for consistent naming
                title = "%s_%s_%s" % (ID, sample, subsample)
                if notes != '':
                    title += "_" + notes
                # create Measurement instance from imported data
                data[str(ID)] = Measurement(
                    ID=str(ID),
                    title=title,
                    filename=filename,
                    sample=sample,
                    subsample=subsample,
                    notes=notes,
                    x=x[check],
                    y=y[check,:],
                    ykey=mode,
                    ylabel=ylabel,
                    technique='FTIR',
                    generate_average=False,
                    spec_type=spec_type,
                    points=points,
                    Fig_dir=Fig_dir,
                    Out_dir=Out_dir,
                    output_folder=''
                )
                if mode == 'T':
                    # add absorbance spectrum
                    data[str(ID)].add_spectrum(
                        'A',
                        transmittance2absorbance(y[check,:]),
                        label="Absorbance (arb. u.)"
                    )
                else:
                    # add transmittance spectrum
                    data[str(ID)].add_spectrum(
                        'T',
                        absorbance2transmittance(y[check,:]),
                        label="Transmittance (%)"
                    )
                
                print("    imported successfully!")
                break
        except Exception as e:
            print("    something went wrong! Exception:", e)
            break



In [ ]:
debug = False

# report which files were imported and which were not
print()
print("%s/%s files imported" % (len(data.keys()), len(spec_dirs)))
for num in Spec_IDs:
    if num in data.keys():
        print("    ", num, u'\u2713', data[num]['filename'])
    else:
        text = ''
        if num in Do_Not_Import:
            text = 'ID in Do_Not_Import list'
        print("    ", num, 'X', text)

# update list of Spec IDs to only include imported spectra
Spec_IDs = list(data.keys())

samples = np.unique([data[num]['sample'] for num in Spec_IDs])
print()
print("samples in dataset:")
for sample in samples:
    print("    ", sample)

# Plot Results by Sample

In [ ]:
# skip this section?
skip = False

# set whether to print debug messages in this section
debug = True

# keyname for x values to plot ('raman_shift', 'wavelength', 'frequency')
x_key = 'frequency'

# keyname for y values to plot
y_key = 'A'

# label for X axis
x_label = "Frequency (cm$^{-1}$)"

# label for Y axis
y_label = 'Absorbance'

# X range for plotting
x_start, x_end = (400, 4000)

# normalise data before plotting?
normalise = True

# offset spectra by this much (0 for no offset)
offset = 0.

# plot average spectrum as well?
plot_average = True

In [ ]:
if skip == True:
    print("SKIPPING SAMPLE SUMMARIES")
else:
    print("PLOTTING SAMPLE SUMMARIES")

# plot sample spectra
for sample in samples:
    # get list of spec IDs for this sample
    result = [ID for ID, measurement in data.items() if measurement.sample == sample]
    spec_count = len(result)
    print()
    print("%s spectra:" % sample, result)
    
    if offset != 0:
        plt.figure(figsize=(8,2+len(result)))
    else:
        plt.figure(figsize=(8,4))
    plt.title("Sample Spectra\n%s" % sample)
    plt.xlabel(x_label)
    plt.xlim(x_start, x_end)
    if offset > 0:
        plt.ylabel(y_label)
        plt.yticks([])
    else:
        plt.ylabel(y_label)
    
    # plot average spectra
    count = 0
    if spec_count > 1:
        x, y = average_spectra([data[ID] for ID in result], x_key, y_key, start=x_start, end=x_end,
                              normalise=normalise, debug=debug)
        plt.plot(x, y - count*offset, 'k', label='mean', zorder=3)
    
    # plot individual spectra
    for ID in result:
        x, y = get_plot_data(data[ID], x_key, y_key, start=x_start, end=x_end,
                normalise=normalise, debug=debug)
        plt.plot(x, y - count*offset, Colour_List[count % len(Colour_List)], label="%s" % (ID))
        count += 1
        
    plt.legend(loc=1)
    plt.minorticks_on()
    plt.tight_layout()
    plt.show()

# Peak Detection
- searches for local maxima that meet two thresholds:
    1) relative intensity (vs maximum)
    2) signal:noise ratio
- produces figure showing which maxima passed and failed
- also shows any manually specified maxima from Manual_Peaks[spec ID]

In [ ]:
# skip this section?
skip = False

# print debug messages in viewer?
debug = True

# minimum signal:noise ratio threshold
SNR_threshold=10

# minimum intensity threshold relative to max
norm_threshold=0.25

# minimum peak-peak separation
min_sep=20

# keyname of x value parameter to use ('raman_shift', 'wavelength', 'frequency')
x_key = 'frequency'

# keyname of y value parameter to use
y_key = 'A'

# back-up keyname if y_key does not exist
alt_key = 'A'

# keyname for detected peak table
new_key ='detected-peaks'

# show plot in viewer?
show_plot = True

# save plots to file?
save_plot = True

In [ ]:
if skip == True:
    print("SKIPPING AUTOMATIC PEAK DETECTION")
else:
    print("DOING AUTOMATIC PEAK DETECTION")
    
    
    process_count = 1
    for ID, measurement in data.items():
        print()
        print("%s/%s detecting peaks for %s" % (process_count, len(data.keys()), measurement.title))

        if hasattr(measurement, y_key) == True:
            key = y_key
        else:
            key = alt_key

        detect_peaks(measurement, x_key, key, new_key, SNR_threshold=SNR_threshold,
                     norm_threshold=norm_threshold, min_sep=min_sep, show_plot=show_plot, save_plot=save_plot,
                     debug=debug)
        process_count += 1
        print("    done!")

# Compare Samples with the same label
- this section groups spectra according to a specified property and plots comparison figures for each group

In [ ]:
# skip this section?
skip = False

# set whether to print debug messages in this section
debug = False

# keyname for x values to plot ('raman_shift', 'wavelength', 'frequency')
x_key = 'frequency'

# keyname for y values to plot
y_key = 'A'

# label for X axis
x_label = "Frequency (cm$^{-1}$)"

# label for Y axis
y_label = 'Normalised Intensity'

# X range for plotting
x_start, x_end = (800, 1800)

# normalise data before plotting?
normalise = True

# offset spectra by this much (0 for no offset)
offset = 0.

# group spectra by this parameter
grouping = 'notes'

# plot average spectra?
plot_average = True

In [ ]:
if skip == True:
    print("SKIPPING SAMPLE COMPARISON")
else:
    print("PLOTTING SAMPLE COMPARISONS")

    # get groupings
    groups = list(np.unique([measurement[grouping] for ID, measurement in data.items() if hasattr(measurement, grouping)]))
    print()
    print("%s groups found:" % len(groups), groups)
    
    # plot spectra for each group separately
    for group in groups:
        result = [ID for ID, measurement in data.items() if getattr(measurement, grouping, "") == group]
        spec_count = len(result)
        print()
        print("plotting group %s, %s spectra found" % (group, spec_count))
        if debug == True:
            print("    ", result)
        
        count = 0

        if offset != 0:
            plt.figure(figsize=(8,2+len(result)))
        else:
            plt.figure(figsize=(8,4))
        plt.title("Sample Spectra\n%s" % group)
        plt.xlabel(x_label)
        plt.xlim(x_start, x_end)
        if offset > 0:
            plt.ylabel(y_label)
            plt.yticks([])
        else:
            plt.ylabel(y_label)

        # plot average spectra
        count = 0
        if plot_average == True and spec_count > 1:
            x, y = average_spectra([data[ID] for ID in result], x_key, y_key, start=x_start, end=x_end,
                                  normalise=normalise, debug=debug)
            plt.plot(x, y - count*offset, 'k', label='mean', zorder=3)

        # plot individual spectra
        for ID in result:
            # check which key to use
            if hasattr(measurement, y_key) == True:
                key = y_key
            else:
                key = alt_key
            
            # get data
            x, y = get_plot_data(data[ID], x_key, key, start=x_start, end=x_end,
                    normalise=normalise, debug=debug)
            
            # plot spectra
            if len(result) < 10:
                # plot individual spectra as distinct lines with their own labels
                plt.plot(x, y - count*offset, Colour_List[count % len(Colour_List)],
                         alpha=1./np.sqrt(spec_count), label="%s" % (ID))
            else:
                # too many spectra to label individually, plot all spectra as semi-transparent lines of the same colour
                plt.plot(x, y - count*offset, Colour_List[groups.index(group) % len(Colour_List)],
                         alpha=1./np.sqrt(spec_count))
            count += 1
            
        plt.legend(loc=1)
        plt.minorticks_on()
        plt.tight_layout()
        if save_plot == True:
            plt.savefig("%s%s-spectra.png" % (Fig_dir, group), dpi=300)
        if show_plot == True:
            plt.show()
        else:
            plt.close()

# Compare Samples with different labels
- this section groups spectra according to a specified property and plots comparison figures for group averages

In [ ]:
# skip this section?
skip = False

# set whether to print debug messages in this section
debug = False

# keyname for x values to plot ('raman_shift', 'wavelength', 'frequency')
x_key = 'frequency'

# keyname for y values to plot
y_key = 'A'

# label for X axis
x_label = "Frequency (cm$^{-1}$)"

# label for Y axis
y_label = 'Normalised Intensity'

# X range for plotting
x_start, x_end = (800, 1800)

# normalise data before plotting?
normalise = True

# offset spectra by this much (0 for no offset)
offset = 0.

# group spectra by this parameter
grouping = 'notes'

# plot average spectra?
plot_average = True

In [ ]:
if skip == True:
    print("SKIPPING SAMPLE COMPARISON")
else:
    print("PLOTTING SAMPLE COMPARISONS")

    # get groupings
    groups = list(np.unique([measurement[grouping] for ID, measurement in data.items() if hasattr(measurement, grouping)]))
    print()
    print("%s groups found:" % len(groups), groups)
        
    if offset != 0:
        plt.figure(figsize=(8,2+len(groups)))
    else:
        plt.figure(figsize=(8,4))
    plt.title("Average Spectra\nGrouped by %s" % grouping)
    plt.xlabel(x_label)
    plt.xlim(x_start, x_end)
    if offset != 0:
        plt.ylabel(y_label)
        plt.yticks([])
    else:
        plt.ylabel(y_label)
        
    count = 0

    # plot spectra for each group separately
    for group in groups:
        result = [ID for ID, measurement in data.items() if getattr(measurement, grouping, "") == group]
        spec_count = len(result)
        
        x, y = average_spectra([data[ID] for ID in result], x_key, y_key, start=x_start, end=x_end,
                    normalise=normalise, debug=debug)
        plt.plot(x, y - count*offset, Colour_List[count % len(Colour_List)], label=group, zorder=3)
        count += 1

    plt.legend(loc=1)
    plt.minorticks_on()
    plt.tight_layout()
    if save_plot == True:
        plt.savefig("%s%s_av-spectra.png" % (Fig_dir, grouping), dpi=300)
    if show_plot == True:
        plt.show()
    else:
        plt.close()

# Save Processed Spectra
- for all measurements, the average spectrum is saved to _av-spectrum.csv
    - includes columns for raw intensity, baselined intensity, and normalised values
- for multi-spec measurements (e.g. maps and line-scans), all point-spectra are saved to _all-spectra_baselined.csv
    - each spectrum is saved as a column along with its X,Y coordinates

In [ ]:
# skip this section?
skip = False

# set whether to print debug messages in this section
debug = False

# set whether to output metadata files
Save_metadata = True

if skip == True:
    print("SKIPPING FILE OUTPUT")
else:
    print("SAVING PROCESSED SPECTRA TO OUTPUT FOLDER")
    
    for ID, measurement in data.items():
        title = measurement.title
        print()
        print(title)

        # one column for each modification of spectrum
        headers = ['Wavelength (nm)', 'Frequency (cm-1)', 'Transmittance (%)', 'Absorbance (Arb. Units)']
        keys = ['wavelength', 'frequency', 'T', 'A']
        save_measurement(measurement, keys=keys, headers=headers, save_name='av-spectrum', debug=debug)

        # save all point-spectra to file (maps & multi-spec files only)
        if measurement.points > 1 and hasattr(measurement, 'y'):
            # one column per baselined point-spectrum
            headers = ['Wavelength (nm)', 'Frequency', 'Absorbance (Arb. Units)']
            keys = ['wavelength', 'frequency', 'A']
            save_measurement(measurement, keys=keys, headers=headers, save_name='all-spectra-baselined', debug=debug)
        print("    %s saved!" % ID)

In [ ]:
print("SCRIPT COMPLETE!")